<h2>Conteo de palabras</h2>

Sobre la base de datos que hemos desarrollado vamos a realizar ciertos análisis mediante el conteo de palabras                                                                 
teniendo en cuenta tanto la fecha de cada intervención, como el grupo político y el género de cada diputado.

Los análisis que queremos realizar son:
    <ul>
        <li>La prevalencia de términos relacionados con la crispación, el odio y el conflicto.</li>
        <li>La prevalencia de ciertos térmnos femeninos o epicenos frente a su masculino genérico.</li>
    </ul>

Los métodos que vamos a desarrollar para ello son:
    <ul>
        <li>El conteo de un grupo de palabras.</li>
        <li>El conteo de palabras prefijadas o sufijadas por un cierto morfema.</li>
        <li>El conteo de un grupo de palabras respecto a otro.</li>
    </ul>

In [1]:
import pandas as pd

df = pd.read_csv(r"")

In [2]:
df

,text,group,gender,year,mistral_large
0,"Señoras y señores Diputados, se abre la sesión...",PP,Male,1979,0
1,Según la relación que consta en la Secretaría ...,PP,Male,1979,0
2,"Siguiendo el orden del día, ruego al señor Esp...",PSOE,Male,1979,0
3,Señoras y señores Diputados. en nombre de la M...,PSOE,Male,1979,0
4,"Al reanudarse la sesión, procedería dar lectur...",PSOE,Male,1979,0
...,...,...,...,...,...
144204,"Muchas gracias, señora presidenta. Buenas tard...",VOX,Male,2024,15
144205,"Muchas gracias, presidenta. Buenas tardes, señ...",PP,Male,2024,30
144206,"Buenas tardes, noches ya, señorías. Probableme...",IU-PODEMOS,Male,2024,22
144207,"Gracias, presidenta. Buenas tardes, señorías. ...",PSOE,Female,2024,9


<h4>Agrupaciones políticas</h4>

Distingimos las diferentes formas de agrupar todos los grupos políticos presentes.

In [3]:
partidos = ["VOX", "PP", "PSOE", "IU-PODEMOS"]
partidos_hombres = [agrupacion + " - Male" for agrupacion in partidos]
partidos_mujeres = [agrupacion + " - Female" for agrupacion in partidos]
indexes = []

for a in partidos:
    indexes.append(a)
for a in partidos_hombres:
    indexes.append(a)
for a in partidos_mujeres:
    indexes.append(a)

In [4]:
years = []
year = 1979
while (year <= 2024):
    years.append(year)
    year+=1

In [5]:
from collections import Counter
import spacy

nlp = spacy.load("es_core_news_lg")

<h3>Distribución de frecuencias</h3>

Organizamos los datos de la siguiente manera:

    freq(Grupo político, Legislatura, Palabra) -> Frecuencia

    word_number(Grupo político - default=Todos, Legislatura) -> Número

In [6]:
%%time
list = []
text = df.loc[23, "text"].lower()
doc = nlp(text)
for token in doc:
    if not token.is_punct and not token.is_space:
        # Convertir a texto en minúsculas y agregar al diccionario
        list.append(str(token))

CPU times: total: 0 ns
Wall time: 7.5 ms


In [13]:
words = {}
for year in years:
    words[year] = {}
    for index in indexes:
        words[year][index] = []
        
for index, data in df.iterrows():
    text = data["text"].lower()
    doc = nlp(text)
    for token in doc:
        if not token.is_punct and not token.is_space:
            words[data["year"]][f"{data['group']} - {data['gender']}"].append(str(token))
            words[data["year"]][f"{data['group']}"].append(str(token))
    if (index % 10000 == 0):
        print(index / 150000 * 100, "% ANALIZADO")

0.0 % ANALIZADO
6.666666666666667 % ANALIZADO
13.333333333333334 % ANALIZADO
20.0 % ANALIZADO
26.666666666666668 % ANALIZADO
33.33333333333333 % ANALIZADO
40.0 % ANALIZADO
46.666666666666664 % ANALIZADO
53.333333333333336 % ANALIZADO
60.0 % ANALIZADO
66.66666666666666 % ANALIZADO
73.33333333333333 % ANALIZADO
80.0 % ANALIZADO
86.66666666666667 % ANALIZADO
93.33333333333333 % ANALIZADO


In [14]:
freq = {}
for year in years:
    freq[year] = {}
    for index in indexes:
        freq[year][index] = []
for year in words:
    for group_gender in freq[year]:
        freq[year][group_gender] = Counter(words[year][group_gender])

In [21]:
for year in years:
    for index in indexes:
        freq[year][index]["nwords"] = len(words[year][index])

In [29]:
freq[2008]["PP"]["vivienda"]

104

In [30]:
import pickle
dir = r""
with open(dir,"wb") as file:
    pickle.dump(freq, file)

In [20]:
freq[2009]["PP"]["vivienda"]

122

In [15]:
terminos_inclusivos = [["españoles", "españolas"], ["ciudadanos", "ciudadanas"], ["ciudadanos", "ciudadanía"], ["profesores", "profesoras"], ["profesores", "profesorado"], ["alumno", "alumna"], ["alumnos", "alumnas"], ["alumnos", "alumnado"], ["niño", "niña"], ["niños", "niñas"], ["niños", "infancia"], ["jueces", "juezas"], ["jueces", "judicatura"], ["alcaldes", "alcaldesas"], ["diputados", "diputadas"], ["médico", "médica"], ["médicos", "médicas"], ["abogado", "abogada"], ["abogados", "abogadas"], ["padres", "madres"], ["padres", "progenitores"]]